# Install libs

In [1]:
!pip install vibdata==1.1.1 signalAI==0.0.4

# Import Libs

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import numpy.typing as npt
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm

import vibdata.raw as raw_datasets
from vibdata.deep.DeepDataset import DeepDataset, convertDataset
from vibdata.deep.signal.transforms import (
    Sequential,
    SplitSampleRate,
    FeatureExtractor,
    FilterByValue,
)
from vibdata.deep.signal.core import SignalSample

# from signalAI.experiments.torch_data import DeepLearningExperiment
from signalAI.experiments.features_1d import Features1DExperiment
from signalAI.utils.group_dataset import GroupDataset
from signalAI.utils.fold_idx_generator import (
    FoldIdxGeneratorUnbiased,
    FoldIdxGeneratorBiased,
)

class GroupCWRULoad(GroupDataset):
    @staticmethod
    def _assigne_group(sample: SignalSample) -> int:
        return sample["metainfo"]["load"]

class GroupCWRUSeverity(GroupDataset):
    @staticmethod
    def _assigne_group(sample: SignalSample) -> int:
        severity = sample["metainfo"]["fault_size"]

        match severity:
            case 0.0:
                return sample["metainfo"]["load"]
            case 0.007:
                return 0
            case 0.014:
                return 1
            case 0.021:
                return 2
            case 0.028:
                return 3

        return None

'''
class GroupMultiRoundCWRULoad(GroupDataset):
    @staticmethod
    def _assigne_group(sample: SignalSample) -> int:
        sample_metainfo = sample["metainfo"]
        return sample_metainfo["label"].astype(str) + " " + sample_metainfo["load"].astype(int).astype(str)

CLASS_DEF = {0: "N", 1: "O", 2: "I", 3: "R"}
CONDITION_DEF = {"0": "0", "1": "1", "2": "2", "3": "3"}
folds_multiround = FoldIdxGeneratorUnbiased(deep_dataset,
                                    GroupMultiRoundCWRULoad ,
                                    dataset_name="CWRU",
                                    multiround=True,
                                    class_def=CLASS_DEF,
                                    condition_def=CONDITION_DEF).generate_folds()
folds_multiround
'''

'\nclass GroupMultiRoundCWRULoad(GroupDataset):\n    @staticmethod\n    def _assigne_group(sample: SignalSample) -> int:\n        sample_metainfo = sample["metainfo"]\n        return sample_metainfo["label"].astype(str) + " " + sample_metainfo["load"].astype(int).astype(str)\n\nCLASS_DEF = {0: "N", 1: "O", 2: "I", 3: "R"}\nCONDITION_DEF = {"0": "0", "1": "1", "2": "2", "3": "3"}\nfolds_multiround = FoldIdxGeneratorUnbiased(deep_dataset,\n                                    GroupMultiRoundCWRULoad ,\n                                    dataset_name="CWRU",\n                                    multiround=True,\n                                    class_def=CLASS_DEF,\n                                    condition_def=CONDITION_DEF).generate_folds()\nfolds_multiround\n'

# Deep Learning Experiments

## Import CRWU dataset

In [3]:
# Get raw root_dir
raw_root_dir = "../data/raw_data/cwru"
raw_dataset = raw_datasets.CWRU_raw(raw_root_dir, download=True)

## Time domain

### 12k SampleRate

In [4]:
transforms_time = Sequential(
    [
        SplitSampleRate(),
    ]
)
print(transforms_time)

Sequential(transforms=[SplitSampleRate()])


In [5]:
deep_root_dir_time = "../data/deep_data/deep_learning"
deep_dataset_time = convertDataset(raw_dataset,filter=FilterByValue(on_field="sample_rate", values=12000),transforms=transforms_time, dir_path=deep_root_dir_time, batch_size=32)

## Generate Unbiased Folds

In [6]:
folds_singleround_deep = FoldIdxGeneratorUnbiased(deep_dataset_time, GroupCWRULoad , dataset_name="CWRU12k_deep").generate_folds()
folds_singleround_deep

Loading group dataset from: ../data/grouping/groups_CustomGroupCWRU12k_deep.npy


array([0., 0., 0., ..., 3., 3., 3.])

## Generate Biased Folds

In [ ]:
folds_singleround_deep = FoldIdxGeneratorBiased(deep_dataset_time, dataset_name="CWRU12k_deep", n_folds=4).generate_folds()
folds_singleround_deep

## DeepLearning Experiments

### Utils

In [9]:
# vibclassifier/experiments/base.py
from abc import ABC, abstractmethod
import json
from typing import Optional, Dict, Any
from vibdata.raw.base import RawVibrationDataset
from vibdata.deep.signal.transforms import Transform

class Experiment(ABC):
    """Classe base abstrata para todos os experimentos de classificação de vibração."""

    def __init__(
        self,
        name: str,
        description: str,
        dataset: Optional[RawVibrationDataset] = None,
        data_transform: Optional[Transform] = None,
        feature_selector = None,
        model = None
    ):
        """
        Inicializa o experimento.

        Args:
            name: Nome identificador do experimento
            description: Descrição detalhada do experimento
            dataset: Conjunto de dados de vibração
            data_transform: Transformação a ser aplicada nos dados brutos
            data_division_method: Método de divisão dos dados (e.g., 'kfold', 'holdout')
            data_division_params: Parâmetros para o método de divisão
            feature_selector: Seletor de features (para experimentos com extração)
            model: Modelo de machine learning/deep learning
        """
        self.name = name
        self.description = description
        self.dataset = dataset
        self.data_transform = data_transform
        self.feature_selector = feature_selector
        self.model = model

        # Resultados serão armazenados aqui
        self.results = {}

    @abstractmethod
    def prepare_data(self):
        """Prepara os dados para o experimento."""
        pass

    @abstractmethod
    def run(self):
        """Executa o experimento completo."""
        pass

    def save_results(self, filepath: str):
        """Salva os resultados do experimento."""
        # Implementação básica - pode ser extendida
        with open(filepath, 'w') as f:
            json.dump(self.results, f)

    def load_results(self, filepath: str):
        """Carrega resultados de um experimento anterior."""
        with open(filepath, 'r') as f:
            self.results = json.load(f)

    def __str__(self):
        return f"Experiment: {self.name}\nDescription: {self.description}"

In [10]:
# vibclassifier/experiments/deep_torch.py
import os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import numpy as np
from typing import List, Dict, Optional, Tuple, Union
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from signalAI.utils.metrics import calculate_metrics
from signalAI.utils.experiment_result import ExperimentResults, FoldResults
import copy

class TorchVibrationDataset(Dataset):
    """Wrapper to convert dataset samples into Torch tensors."""
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Função auxiliar KL (necessária para SAE)
def kl_divergence(rho, rho_hat):
    rho_hat = torch.mean(rho_hat, dim=0)
    rho = torch.tensor([rho] * len(rho_hat), device=rho_hat.device)
    epsilon = 1e-7
    term1 = rho * torch.log((rho + epsilon) / (rho_hat + epsilon))
    term2 = (1 - rho) * torch.log((1 - rho + epsilon) / (1 - rho_hat + epsilon))
    return torch.sum(term1 + term2)

class DeepLearningExperiment(Experiment):
    def __init__(
        self,
        name: str,
        description: str,
        dataset,
        data_fold_idxs: List[int],
        model: nn.Module,
        criterion: Optional[nn.Module] = None,
        # --- Parâmetros para Autoencoder ---
        reconstruction_criterion: Optional[nn.Module] = None,
        recon_loss_weight: float = 1.0,
        sparsity_target: Optional[float] = None,
        sparsity_weight: float = 0.0,
        pretrain_epochs: int = 0, # Épocas para treino do autoencoder quando aplicavel
        # ----------------------------------------
        optimizer_class: Optional[torch.optim.Optimizer] = optim.Adam,
        batch_size: int = 32,
        lr: float = 1e-3,
        num_epochs: int = 20, # Épocas para treino do classificador
        val_split: float = 0.2,
        output_dir: str = "results_torch",
        device: str = "cuda" if torch.cuda.is_available() else "cpu",
        **kwargs
    ):
        super().__init__(name, description, dataset, model=model, **kwargs)
        self.data_fold_idxs = data_fold_idxs
        self.output_dir = Path(output_dir)
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.pretrain_epochs = pretrain_epochs # Armazena épocas de treino do autoencoder quando aplicavel
        self.val_split = val_split
        self.device = device
        self.optimizer_class = optimizer_class
        self.lr = lr
        self.criterion = criterion if criterion is not None else nn.CrossEntropyLoss()
        
        self.reconstruction_criterion = reconstruction_criterion
        self.recon_loss_weight = recon_loss_weight
        self.sparsity_target = sparsity_target
        self.sparsity_weight = sparsity_weight
        
        self.is_sae_task = self.sparsity_target is not None and self.sparsity_weight > 0.0
        # Define se é tarefa de AE. Se tiver reconstrução OU for SAE.
        self.is_autoencoder_task = reconstruction_criterion is not None or self.is_sae_task or "AE1D" in model.__class__.__name__

        if self.is_sae_task and self.reconstruction_criterion is None:
             print("Warning: SAE task detected but no reconstruction_criterion. Defaulting to MSELoss.")
             self.reconstruction_criterion = nn.MSELoss()

        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs")
            model = torch.nn.DataParallel(model) 
        
        # Guardar referência ao modelo base para acessar sub-módulos (encoder, decoder)
        self.original_model = model.module if isinstance(model, nn.DataParallel) else model

        self.n_outer_folds = len(np.unique(data_fold_idxs))
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.prepare_data()

    def prepare_data(self):
        features, labels = [], []
        for sample in self.dataset:
            features.append(sample['signal'][0]) 
            labels.append(sample['metainfo']['label'])
        self.X = np.array(features)
        self.y = np.array(labels)

    def _train_one_fold(
        self, X_train, y_train, X_test, y_test, fold_idx: int
    ) -> FoldResults:
        # --- Setup de Dados ---
        train_dataset = TorchVibrationDataset(X_train, y_train)
        test_dataset = TorchVibrationDataset(X_test, y_test)
        val_size = int(self.val_split * len(train_dataset))
        train_size = len(train_dataset) - val_size
        train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
        
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

        # Copia o modelo para este fold
        model = copy.deepcopy(self.model.to(self.device))
        
        # Acessa o modelo "real" por trás do DataParallel se necessário para verificar atributos
        model_core = model.module if isinstance(model, nn.DataParallel) else model

        # Verifica se o modelo tem a estrutura de AE (encoder, decoder, classifier)
        has_ae_structure = hasattr(model_core, 'encoder') and hasattr(model_core, 'decoder') and hasattr(model_core, 'classifier')

        # ============================================================
        # 1. TREINO DO AUTOENCODER (Apenas se for AE e pretrain_epochs > 0)
        # ============================================================
        if self.is_autoencoder_task and self.pretrain_epochs > 0 and has_ae_structure:
            print(f"[Fold {fold_idx}] AutoEncoder training ({self.pretrain_epochs} epochs)...")
            
            # Otimizador apenas para Encoder + Decoder
            optimizer_ae = self.optimizer_class([
                {'params': model_core.encoder.parameters()},
                {'params': model_core.decoder.parameters()}
            ], lr=self.lr)

            for epoch in range(self.pretrain_epochs):
                model.train()
                running_recon_loss = 0.0
                
                for xb, _ in train_loader: # Ignora labels (y)
                    xb = xb.to(self.device)
                    input_data = xb # Cópia para loss

                    # Ajuste de shape (1D/2D Conv)
                    if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                        xb = xb.unsqueeze(1)
                    elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                         side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                    optimizer_ae.zero_grad()
                    outputs = model(xb) # Retorna (class, recon, [sparsity])

                    if isinstance(outputs, tuple):
                        # Ignora classificação, foca na reconstrução
                        reconstruction = outputs[1] 
                        
                        loss = self.reconstruction_criterion(reconstruction, input_data)
                        
                        # Adiciona esparsidade se for SAE
                        if self.is_sae_task and len(outputs) > 2:
                            latent_features = outputs[2]
                            loss += self.sparsity_weight * kl_divergence(self.sparsity_target, latent_features)
                        
                        loss.backward()
                        optimizer_ae.step()
                        running_recon_loss += loss.item() * input_data.size(0)
                
                avg_recon_loss = running_recon_loss / len(train_loader.dataset)
                if (epoch + 1) % 5 == 0 or epoch == 0:
                    print(f"  [Pre-train] Epoch {epoch+1}/{self.pretrain_epochs} Recon Loss: {avg_recon_loss:.4f}")

        # ============================================================
        # 2. TREINO DO CLASSIFICADOR (Ou treino padrão para MLP e CNN)
        # ============================================================
        print(f"[Fold {fold_idx}] Classifier training ({self.num_epochs} epochs)...")

        # Define otimizador para a fase supervisionada
        if has_ae_structure and self.is_autoencoder_task:
            # Se for AE: Treina Encoder + Classifier (Decoder congelado ou ignorado pelo otimizador)
            optimizer_clf = self.optimizer_class([
                {'params': model_core.encoder.parameters()},
                {'params': model_core.classifier.parameters()}
            ], lr=self.lr)
        else:
            # Se for MLP/CNN padrão: Treina todos os parâmetros
            optimizer_clf = self.optimizer_class(model.parameters(), lr=self.lr)

        train_losses, val_losses = [], []
        
        for epoch in range(self.num_epochs):
            epoch_start = time.time()
            model.train()
            running_loss = 0.0
            
            for xb, yb in train_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                
                # Ajuste de shape
                if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                     xb = xb.unsqueeze(1)
                elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                     side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                optimizer_clf.zero_grad()
                outputs = model(xb)

                # Cálculo da perda apenas de CLASSIFICAÇÃO
                if isinstance(outputs, tuple):
                    classification_output = outputs[0] # Pega apenas a classificação
                else:
                    classification_output = outputs # Modelo padrão

                loss = self.criterion(classification_output, yb)
                
                loss.backward()
                optimizer_clf.step()
                running_loss += loss.item() * xb.size(0)

            avg_train_loss = running_loss / len(train_loader.dataset)

            # --- Validação ---
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(self.device), yb.to(self.device)
                    # Ajuste de shape (mesmo do treino)
                    if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                         xb = xb.unsqueeze(1)
                    elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                         side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                    outputs = model(xb)
                    
                    if isinstance(outputs, tuple):
                        classification_output = outputs[0]
                    else:
                        classification_output = outputs

                    loss = self.criterion(classification_output, yb)
                    val_loss += loss.item() * xb.size(0)

            avg_val_loss = val_loss / len(val_loader.dataset)
            
            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)
            
            epoch_time = time.time() - epoch_start
            if (epoch + 1) % 5 == 0 or epoch == 0:
                print(f"  [Supervised] Epoch {epoch+1}/{self.num_epochs} Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Time: {epoch_time:.2f}s")

        # Salvar curvas e checkpoint
        plt.figure()
        plt.plot(train_losses, label="Train Loss (Clf)")
        plt.plot(val_losses, label="Val Loss (Clf)")
        plt.legend(); plt.title(f"Loss Curve - Fold {fold_idx}")
        plt.savefig(os.path.join(self.dir_path, f"loss_curve_fold{fold_idx}_{self.start_time}.png")); plt.close()
        
        torch.save(model_core.state_dict(), os.path.join(self.dir_path, f"model_fold{fold_idx}.pt"))

        # --- Teste Final ---
        y_true, y_pred, y_proba = [], [], []
        model.eval()
        with torch.no_grad():
            for xb, yb in test_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                if any(isinstance(m, nn.Conv1d) for m in model.modules()) and xb.ndim == 2:
                     xb = xb.unsqueeze(1)
                elif any(isinstance(m, nn.Conv2d) for m in model.modules()) and xb.ndim == 2:
                     side = int(np.sqrt(xb.shape[1])); xb = xb.view(xb.size(0), 1, side, side)

                outputs = model(xb)
                if isinstance(outputs, tuple):
                    classification_output = outputs[0]
                else:
                    classification_output = outputs

                probs = torch.softmax(classification_output, dim=1)
                preds = torch.argmax(probs, dim=1)
                y_true.extend(yb.cpu().numpy()); y_pred.extend(preds.cpu().numpy()); y_proba.extend(probs.cpu().numpy())

        metrics = calculate_metrics(np.array(y_true), np.array(y_pred), np.array(y_proba))
        return FoldResults(fold_idx, np.array(y_true), np.array(y_pred), np.array(y_proba), metrics)

    def run(self) -> ExperimentResults:
        self.start_time = time.strftime("%Y%m%d_%H%M%S")
        self.dir_path = os.path.join(self.output_dir, f"results_{self.name}_{self.start_time}")
        os.makedirs(self.dir_path, exist_ok=True)

        results = ExperimentResults(
            experiment_name=self.name, description=self.description,
            model_name=self.original_model.__class__.__name__, feature_names=None,
            config={'n_outer_folds': self.n_outer_folds, 'pretrain_epochs': self.pretrain_epochs, 
                    'finetune_epochs': self.num_epochs, 'batch_size': self.batch_size, 'lr': self.lr}
        )

        for outer_fold in range(self.n_outer_folds):
            print(f"\n=== Outer Fold {outer_fold+1}/{self.n_outer_folds} ===")
            train_mask = self.data_fold_idxs != outer_fold
            test_mask = self.data_fold_idxs == outer_fold
            
            try:
                fold_result = self._train_one_fold(self.X[train_mask], self.y[train_mask], self.X[test_mask], self.y[test_mask], outer_fold)
                results.add_fold_result(fold_result)
                print(f"  Result: Acc={fold_result.metrics['accuracy']:.4f}, F1={fold_result.metrics['f1']:.4f}")
            except Exception as e:
                print(f"Error in fold {outer_fold}: {e}")
                import traceback; traceback.print_exc()

        results.calculate_overall_metrics()
        results.save_json(os.path.join(self.dir_path, f"results.json"))
        print("\n=== Final Results ===")
        print(f"Mean Accuracy: {results.overall_metrics['accuracy']:.4f}")
        return results

### MLP

#### Adaptação com mais camadas

In [ ]:
class MLP1D(nn.Module):
    """
    Implementação do MLP baseado na Figura 2 do artigo
    Projetado para entrada 1D com 1024 features.
    """
    def __init__(self, input_length: int, num_classes: int, dropout_rate: float = 0.5):
        super(MLP1D, self).__init__()

        # O artigo especifica 5 camadas FC+BN, mas como a entrada é de tamanho 12000 são utilizadas mais camadas
        # Os tamanhos de feature são: 12000 -> 8192 -> 4096 -> 2048 -> 1024 -> 512 -> 256 -> 128 -> 64
        self.layers = nn.Sequential(
            nn.Linear(input_length, 8192),
            nn.BatchNorm1d(8192),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(8192, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(4096, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        self.classifier = nn.Linear(64, num_classes)

    def forward(self, x):
        features = self.layers(x)
        output = self.classifier(features)
        return output

In [ ]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = MLP1D(input_length=input_length, num_classes=num_classes, dropout_rate=0.5)

exp = DeepLearningExperiment(
    name="mlp1d_vibration",
    description="1D MLP for vibration signals",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,  # numpy array of fold indices
    model=model,
    batch_size=2048,
    lr=1e-3,
    num_epochs=10
)

results = exp.run()

#### Arquitetura original do artigo adicinando dropout

In [11]:
class MLP1D_paper(nn.Module):
    """
    Implementação de um MLP baseline razoável para entrada de 12000.
    A arquitetura segue a Figura 2 do artigo,
    adaptando apenas a primeira camada e adicionando Dropout.
    """
    def __init__(self, input_length: int = 12000, num_classes: int = 4, dropout_rate: float = 0.4):
        super(MLP1D_paper, self).__init__()

        # Arquitetura de 5 camadas ocultas inspirada no artigo
        # 12000 -> 1024 -> 512 -> 256 -> 128 -> 64
        self.layers = nn.Sequential(
            # 1. Camada de entrada adaptada
            nn.Linear(input_length, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate), # Dropout para regularizar a 1ª camada grande

            # 2. Camadas ocultas seguintes
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # 3.
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # 4.
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # 5. Camada final de features
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        # Classificador separado
        self.classifier = nn.Linear(64, num_classes)

    def forward(self, x):
        # x deve ter a forma [batch_size, input_length]
        features = self.layers(x)
        output = self.classifier(features)
        return output

In [12]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = MLP1D_paper(input_length=input_length, num_classes=num_classes, dropout_rate=0.5)

exp = DeepLearningExperiment(
    name="mlp1d_vibration",
    description="1D MLP for vibration signals",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,  # numpy array of fold indexes
    model=model,
    batch_size=2048,
    lr=1e-3,
    pretrain_epochs=0,
    num_epochs=10
)

results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4134, Val Loss: 1.4004, Time: 2.75s
  [Supervised] Epoch 5/10 Train Loss: 1.3596, Val Loss: 1.3938, Time: 1.79s
  [Supervised] Epoch 10/10 Train Loss: 1.3342, Val Loss: 1.3763, Time: 1.79s
  Result: Acc=0.2267, F1=0.2239

=== Outer Fold 2/4 ===
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4062, Val Loss: 1.3935, Time: 1.78s
  [Supervised] Epoch 5/10 Train Loss: 1.3565, Val Loss: 1.3879, Time: 1.78s
  [Supervised] Epoch 10/10 Train Loss: 1.3228, Val Loss: 1.3724, Time: 2.75s
  Result: Acc=0.3422, F1=0.2591

=== Outer Fold 3/4 ===
[Fold 2] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4214, Val Loss: 1.3964, Time: 2.06s
  [Supervised] Epoch 5/10 Train Loss: 1.3670, Val Loss: 1.3914, Time: 1.80s
  [Supervised] Epoch 10/10 Train Loss: 1.3344, Val Loss: 1.3761, Time: 1.75s
  Result:

### 1D AutoEncoder

In [13]:
class AE1D(nn.Module):
    """
    Implementação de um Autoencoder 1D com Dropout para regularização,
    adaptado para uma entrada de comprimento 12000 e incluindo um classificador.
    """
    def __init__(self, input_length: int = 12000, latent_dim: int = 64, num_classes: int = 4, dropout_rate: float = 0.4):
        """
        Args:
            input_length: Comprimento do sinal de entrada.
            latent_dim: Dimensão do espaço latente.
            num_classes: Número de classes para classificação.
            dropout_rate: Probabilidade de um neurônio ser zerado (ex: 0.4 = 40%).
        """
        super(AE1D, self).__init__()
        self.input_length = input_length
        self.latent_dim = latent_dim

        # --- Encoder ---
        # Adicionadas camadas nn.Dropout após cada nn.ReLU
        self.encoder = nn.Sequential(
            nn.Linear(input_length, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            # Camada final do encoder para o espaço latente
            nn.Linear(128, latent_dim)
        )

        # --- Decoder ---
        # Adicionadas camadas nn.Dropout após cada nn.ReLU
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            # Camada final do decoder (sem dropout)
            nn.Linear(1024, input_length)
        )

        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        """
        Define o passo forward. (Permanece inalterado)
        A entrada x deve ter a forma [batch_size, input_length].
        """
        # Codifica a entrada para o espaço latente
        latent_features = self.encoder(x)

        # Decodifica do espaço latente para reconstruir a entrada
        reconstruction = self.decoder(latent_features)

        # Classifica a partir do espaço latente
        classification_output = self.classifier(latent_features)

        # Retorna a saída da classificação e a reconstrução
        return classification_output, reconstruction

In [14]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = AE1D(input_length=input_length, latent_dim=64, num_classes=num_classes, dropout_rate=0.5)

# Defina os critérios
classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

exp = DeepLearningExperiment(
    name="ae1d_vibration_combined_loss",
    description="1D AE for vibration signals (Combined Loss)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # critério para treinamento do autoencoder
    criterion=classification_criterion,                # critério para treinamento do classificador
    pretrain_epochs=10,  # Treina Encoder+Decoder apenas com MSELoss
    num_epochs=10,       # Treina Encoder+Classifier apenas com CrossEntropy
    recon_loss_weight=1.0, # recon_loss_weight pode ser 1.0 (pois é a única loss na fase 1)
    batch_size=2048,
    lr=1e-4
)

results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.4713
  [Pre-train] Epoch 5/10 Recon Loss: 0.3989
  [Pre-train] Epoch 10/10 Recon Loss: 0.3488
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4552, Val Loss: 1.4104, Time: 2.78s
  [Supervised] Epoch 5/10 Train Loss: 1.4194, Val Loss: 1.4049, Time: 2.61s
  [Supervised] Epoch 10/10 Train Loss: 1.3842, Val Loss: 1.3987, Time: 2.62s
  Result: Acc=0.0900, F1=0.0394

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.4730
  [Pre-train] Epoch 5/10 Recon Loss: 0.3958
  [Pre-train] Epoch 10/10 Recon Loss: 0.3459
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4562, Val Loss: 1.4086, Time: 4.02s
  [Supervised] Epoch 5/10 Train Loss: 1.4225, Val Loss: 1.4034, Time: 3.50s
  [Supervised] Epoch 10/10 Train Loss: 1.4029, Val Loss: 1.

### 1D Sparse AutoEncoder

In [15]:
class SAE1D(nn.Module):
    """
    Implementação de um Sparse Autoencoder 1D baseado em camadas lineares,
    adaptado para uma entrada de comprimento 12000 e incluindo um classificador.
    A esparsidade é aplicada externamente via perda KL na camada latente.
    """
    def __init__(self, input_length: int = 12000, latent_dim: int = 64, num_classes: int = 4):
        """
        Args:
            input_length: Comprimento do sinal de entrada.
            latent_dim: Dimensão do espaço latente (onde a esparsidade será aplicada).
            num_classes: Número de classes para classificação.
        """
        super(SAE1D, self).__init__()
        self.input_length = input_length
        self.latent_dim = latent_dim

        self.encoder = nn.Sequential(
            nn.Linear(input_length, 1024), nn.BatchNorm1d(1024), nn.ReLU(inplace=True),
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(inplace=True),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(inplace=True),
            nn.Linear(128, latent_dim)
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128), nn.BatchNorm1d(128), nn.ReLU(inplace=True),
            nn.Linear(128, 256), nn.BatchNorm1d(256), nn.ReLU(inplace=True),
            nn.Linear(256, 512), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 1024), nn.BatchNorm1d(1024), nn.ReLU(inplace=True),
            nn.Linear(1024, input_length)
        )

        self.classifier = nn.Linear(latent_dim, num_classes)

        self.sparsity_activation = nn.Sigmoid()

    def forward(self, x):
        """
        Define o passo forward.
        Retorna: (saída da classificação, reconstrução, ativações latentes para esparsidade)
        """
        latent_features_raw = self.encoder(x) # Saída linear do encoder

        # Aplica Sigmoid para o cálculo da esparsidade
        latent_features_for_sparsity = self.sparsity_activation(latent_features_raw)

        reconstruction = self.decoder(latent_features_raw) # Decoder usa a saída linear
        classification_output = self.classifier(latent_features_raw) # Classifier usa a saída linear

        # Retorna as 3 componentes necessárias
        return classification_output, reconstruction, latent_features_for_sparsity

In [16]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")

model = SAE1D(input_length=input_length, latent_dim=64, num_classes=num_classes)

# loss criterions
classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

exp = DeepLearningExperiment(
    name="sae1d_vibration_sequential",
    description="1D Sparse AE for vibration signals (Sequential Training)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # ae train
    criterion=classification_criterion,                # classifier train
    pretrain_epochs=10,  # Treina Encoder+Decoder com MSE + Esparsidade KL
    num_epochs=10,       # Treina Encoder+Classifier com CrossEntropy
    sparsity_target=0.05,  # (Rho) Nível de esparsidade desejado (ex: 5% dos neurônios ativos) 
    sparsity_weight=0.1,   # (Beta) Peso da penalidade KL na perda total
    recon_loss_weight=1.0, # Peso da reconstrução (geralmente 1.0 na fase de pré-treino puro)
    batch_size=2048,
    lr=1e-3
)

results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 3.4225
  [Pre-train] Epoch 5/10 Recon Loss: 2.8151
  [Pre-train] Epoch 10/10 Recon Loss: 2.0672
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4018, Val Loss: 1.3597, Time: 2.59s
  [Supervised] Epoch 5/10 Train Loss: 1.0005, Val Loss: 1.2866, Time: 2.47s
  [Supervised] Epoch 10/10 Train Loss: 0.3865, Val Loss: 1.1275, Time: 2.46s
  Result: Acc=0.3933, F1=0.3653

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 3.4422
  [Pre-train] Epoch 5/10 Recon Loss: 2.8385
  [Pre-train] Epoch 10/10 Recon Loss: 2.1083
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.3926, Val Loss: 1.3684, Time: 3.04s
  [Supervised] Epoch 5/10 Train Loss: 1.0002, Val Loss: 1.2828, Time: 3.49s
  [Supervised] Epoch 10/10 Train Loss: 0.4411, Val Loss: 1.

### 1D Denoising AutoEncoder

In [17]:
class DAE1D(nn.Module):
    """
    Implementação de um Denoising Autoencoder (DAE) 1D.
    Adaptação para entrada de comprimento 12000 e incluindo um classificador.
    O DAE adiciona ruído Gaussiano à entrada durante o treinamento para aprender a reconstruir sinais limpos.
    """
    def __init__(self, input_length: int = 12000, latent_dim: int = 64, num_classes: int = 4, noise_factor: float = 0.5):
        """
        Args:
            input_length: Comprimento do sinal de entrada.
            latent_dim: Dimensão do espaço latente.
            num_classes: Número de classes para classificação.
            noise_factor: Fator de intensidade do ruído Gaussiano (desvio padrão).
        """
        super(DAE1D, self).__init__()
        self.input_length = input_length
        self.latent_dim = latent_dim
        self.noise_factor = noise_factor

        # --- Encoder ---
        # Recebe a entrada (potencialmente ruidosa) e comprime
        self.encoder = nn.Sequential(
            nn.Linear(input_length, 1024), nn.BatchNorm1d(1024), nn.ReLU(inplace=True),
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(inplace=True),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(inplace=True),
            nn.Linear(128, latent_dim) 
        )

        # --- Decoder ---
        # Tenta reconstruir o sinal limpo a partir do latente
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128), nn.BatchNorm1d(128), nn.ReLU(inplace=True),
            nn.Linear(128, 256), nn.BatchNorm1d(256), nn.ReLU(inplace=True),
            nn.Linear(256, 512), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 1024), nn.BatchNorm1d(1024), nn.ReLU(inplace=True),
            nn.Linear(1024, input_length)
        )

        # --- Classifier ---
        # Classifica com base nas features latentes
        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, x):
        """
        Define o passo forward.
        Args:
            x: Entrada limpa (batch_size, input_length)
        Returns:
            classification_output: Logits para classificação
            reconstruction: Sinal reconstruído (tentativa de remover ruído)
        """
        
        # adiciona ruído gaussiano apenas durante o treinamento
        if self.training:
            noise = torch.randn_like(x) * self.noise_factor
            x_noisy = x + noise
        else:
            x_noisy = x

        # Encoder processa a entrada com ruído ("input + noise")
        latent_features = self.encoder(x_noisy)

        # Decoder tenta reconstruir a entrada limpa
        reconstruction = self.decoder(latent_features)

        # Classificador usa as features latentes
        classification_output = self.classifier(latent_features)

        return classification_output, reconstruction

In [18]:
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")

# noise_factor controla o ruído
model = DAE1D(input_length=input_length, latent_dim=64, num_classes=num_classes, noise_factor=0.5)

classification_criterion = nn.CrossEntropyLoss()
reconstruction_criterion = nn.MSELoss()

# 4. Configuração do Experimento
exp = DeepLearningExperiment(
    name="dae1d_vibration_sequential",
    description="1D Denoising AE for vibration signals (Sequential Training)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    reconstruction_criterion=reconstruction_criterion, # ae train
    criterion=classification_criterion,                # classifier train
    pretrain_epochs=10,  # Treina Encoder+Decoder com MSELoss (DAE)
    num_epochs=10,       # Treina Encoder+Classifier com CrossEntropy
    recon_loss_weight=1.0,
    batch_size=2048, # Batch grande ajuda na estabilidade do AE
    lr=1e-3
)

# 5. Execução
results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.2937
  [Pre-train] Epoch 5/10 Recon Loss: 0.1457
  [Pre-train] Epoch 10/10 Recon Loss: 0.1320
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4190, Val Loss: 1.3685, Time: 3.10s
  [Supervised] Epoch 5/10 Train Loss: 1.3047, Val Loss: 1.3443, Time: 2.85s
  [Supervised] Epoch 10/10 Train Loss: 1.0209, Val Loss: 1.3088, Time: 2.76s
  Result: Acc=0.3667, F1=0.2589

=== Outer Fold 2/4 ===
[Fold 1] AutoEncoder training (10 epochs)...
  [Pre-train] Epoch 1/10 Recon Loss: 0.3011
  [Pre-train] Epoch 5/10 Recon Loss: 0.1530
  [Pre-train] Epoch 10/10 Recon Loss: 0.1396
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4210, Val Loss: 1.3639, Time: 2.71s
  [Supervised] Epoch 5/10 Train Loss: 1.2964, Val Loss: 1.3382, Time: 3.20s
  [Supervised] Epoch 10/10 Train Loss: 1.0024, Val Loss: 1.

### 1D CNN (Exemplo aula)

In [19]:
class CNN1D(nn.Module):
    def __init__(self, input_length: int, num_classes: int):
        """
        Simple 1D CNN for vibration signal classification.

        Args:
            input_length: length of the input signal
            num_classes: number of output classes
        """
        super(CNN1D, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, padding=3)
        self.bn1 = nn.BatchNorm1d(16)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(32)
        self.pool2 = nn.MaxPool1d(2)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.pool3 = nn.AdaptiveMaxPool1d(16)  # reduce dynamically to fixed size

        # compute flattened size
        example_input = torch.zeros(1, 1, input_length)  # [B, C, L]
        with torch.no_grad():
            x = self.pool1(F.relu(self.bn1(self.conv1(example_input))))
            x = self.pool2(F.relu(self.bn2(self.conv2(x))))
            x = self.pool3(F.relu(self.bn3(self.conv3(x))))
            flattened_size = x.shape[1] * x.shape[2]

        self.fc1 = nn.Linear(flattened_size, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # x shape: [B, L] or [B, 1, L]
        if x.ndim == 2:
            x = x.unsqueeze(1)  # add channel dim

        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [20]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = CNN1D(input_length=input_length, num_classes=num_classes)

exp = DeepLearningExperiment(
    name="cnn1d_vibration",
    description="1D CNN for vibration signals",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,  # numpy array of fold indexes
    model=model,
    batch_size=64,
    lr=1e-3,
    num_epochs=10
)

results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.3666, Val Loss: 1.2145, Time: 43.26s
  [Supervised] Epoch 5/10 Train Loss: 0.6495, Val Loss: 0.5737, Time: 43.67s
  [Supervised] Epoch 10/10 Train Loss: 0.4985, Val Loss: 0.4269, Time: 62.42s
  Result: Acc=0.7033, F1=0.7032

=== Outer Fold 2/4 ===
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4682, Val Loss: 1.2702, Time: 44.80s
  [Supervised] Epoch 5/10 Train Loss: 0.7290, Val Loss: 0.6640, Time: 42.27s
  [Supervised] Epoch 10/10 Train Loss: 0.6010, Val Loss: 0.5092, Time: 39.85s
  Result: Acc=0.8406, F1=0.8370

=== Outer Fold 3/4 ===
[Fold 2] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.3810, Val Loss: 1.2470, Time: 38.03s
  [Supervised] Epoch 5/10 Train Loss: 0.6738, Val Loss: 0.6343, Time: 38.69s
  [Supervised] Epoch 10/10 Train Loss: 0.5085, Val Loss: 0.6051, Time: 39.55s


### 1D CNN (7 layers)

In [21]:
class CNN1D_7Layers(nn.Module):
    """
    Implementação da 1D CNN (7 layers) baseada no artigo.
    Estrutura:
    - Bloco 1: 2x(Conv + BN) + MaxPool
    - Bloco 2: 2x(Conv + BN) + AdaptiveMaxPool
    - Classificador: 3x FC (Fully Connected)
    """
    def __init__(self, input_length: int = 1024, num_classes: int = 10):
        super(CNN1D_7Layers, self).__init__()
        
        # block 1
        self.block1 = nn.Sequential(
            # 1ª Convolução: 1 canal -> 16 canais
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=15, stride=1, padding=0),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            
            # 2ª Convolução: 16 canais -> 32 canais
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),

            # MaxPool: Reduz pela metade a dimensão temporal
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # block 2        
        self.block2 = nn.Sequential(
            # 3ª Convolução: 32 canais -> 64 canais
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            
            # 4ª Convolução: 64 canais -> 128 canais
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            
            # Adaptive MaxPool: Reduz dimensão temporal para tamanho fixo 4
            nn.AdaptiveMaxPool1d(output_size=4)
        )
        
        # classificador
        # Input achatado: 128 canais * 4 dimensão temporal = 512 features
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # 1ª FC: Entrada 512 (4*128) -> Saída 256
            nn.Linear(128 * 4, 256),
            nn.ReLU(inplace=True),
            # O artigo usa Dropout nessas camadas
            nn.Dropout(0.5), 
            
            # 2ª FC: 256 -> 64
            nn.Linear(256, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            # 3ª FC (Saída): 64 -> Num_class
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # Ajuste de forma para garantir [Batch, 1, Length]
        if x.ndim == 2:
            x = x.unsqueeze(1)
            
        # block 1
        x = self.block1(x)
        
        # block 2
        x = self.block2(x)
        
        # classificador
        x = self.classifier(x)
        
        return x

In [22]:
# suppose dataset is already a DeepDataset like before
input_length = deep_dataset_time[0]['signal'][0].shape[-1]
num_classes = len(set([s['metainfo']['label'] for s in deep_dataset_time]))

print(f"Input length: {input_length}, Num classes: {num_classes}")
model = CNN1D_7Layers(input_length=input_length, num_classes=num_classes)

exp = DeepLearningExperiment(
    name="cnn1d_7layers_vibration",
    description="1D CNN (7 Layers) for vibration signals (Standard Supervised)",
    dataset=deep_dataset_time,
    data_fold_idxs=folds_singleround_deep,
    model=model,
    batch_size=64, 
    lr=1e-3,       
    num_epochs=10
)

results = exp.run()

Input length: 12000, Num classes: 4

=== Outer Fold 1/4 ===
[Fold 0] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4664, Val Loss: 1.3428, Time: 146.70s
  [Supervised] Epoch 5/10 Train Loss: 0.9007, Val Loss: 0.7399, Time: 131.51s
  [Supervised] Epoch 10/10 Train Loss: 0.5112, Val Loss: 0.3899, Time: 144.84s
  Result: Acc=0.7433, F1=0.7419

=== Outer Fold 2/4 ===
[Fold 1] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4432, Val Loss: 1.3534, Time: 132.86s
  [Supervised] Epoch 5/10 Train Loss: 0.9813, Val Loss: 0.8117, Time: 150.54s
  [Supervised] Epoch 10/10 Train Loss: 0.5889, Val Loss: 0.4515, Time: 136.91s
  Result: Acc=0.7969, F1=0.7936

=== Outer Fold 3/4 ===
[Fold 2] Classifier training (10 epochs)...
  [Supervised] Epoch 1/10 Train Loss: 1.4548, Val Loss: 1.3399, Time: 131.43s
  [Supervised] Epoch 5/10 Train Loss: 0.9925, Val Loss: 0.8643, Time: 130.60s
  [Supervised] Epoch 10/10 Train Loss: 0.6760, Val Loss: 0.5740, Time: